In [1]:
!pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

abs_diff_plus_log_odds_scores_norm = pd.read_csv("/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/output/SNP_PRVCS_CAN_0_5023-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')
abs_diff_plus_log_odds_scores_norm.head(), abs_diff_plus_log_odds_scores_norm.shape


(   BroadDnd41CtcfUniPk151-ran  BroadDnd41Ezh239875UniPk151-ran  \
 0                    0.306653                        -0.556821   
 1                   -0.111710                        -0.113748   
 2                    0.090462                        -0.502083   
 3                   -0.143991                        -0.349744   
 4                   -0.144492                        -0.409669   
 
    BroadGm12878CtcfUniPk151-ran  BroadGm12878Ezh239875UniPk151-ran  \
 0                      0.018333                          -0.521790   
 1                      0.118644                           1.831607   
 2                     -0.059683                          -0.682626   
 3                     -0.189783                          -0.485974   
 4                     -0.191357                           0.003039   
 
    BroadH1hescChd1a301218aUniPk151-ran  BroadH1hescCtcfUniPk151-ran  \
 0                            -0.397740                     0.304906   
 1                      

In [10]:
import xgboost

In [11]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot

**TRAINING**

In [12]:
X=abs_diff_plus_log_odds_scores_norm
X

,BroadDnd41CtcfUniPk151-ran,BroadDnd41Ezh239875UniPk151-ran,BroadGm12878CtcfUniPk151-ran,BroadGm12878Ezh239875UniPk151-ran,BroadH1hescChd1a301218aUniPk151-ran,BroadH1hescCtcfUniPk151-ran,BroadH1hescEzh239875UniPk151-ran,BroadH1hescJarid1aab26049UniPk151-ran,BroadH1hescRbbp5a300109aUniPk151-ran,BroadHelas3CtcfUniPk151-ran,...,UwMcf7CtcfUniPk151-ran.1,UwNb4CtcfUniPk151-ran.1,UwNhdfneoCtcfUniPk151-ran.1,UwNhekCtcfUniPk151-ran.1,UwNhlfCtcfUniPk151-ran.1,UwRptecCtcfUniPk151-ran.1,UwSaecCtcfUniPk151-ran.1,UwSknshraCtcfUniPk151-ran.1,UwWerirb1CtcfUniPk151-ran.1,UwWi38CtcfUniPk151-ran.1
0,0.306653,-0.556821,0.018333,-0.521790,-0.397740,0.304906,-0.250733,-0.502371,-0.331445,0.104589,...,0.161162,2.493501,0.508397,0.315912,0.458727,0.510215,-0.314536,0.241683,0.519838,1.156624
1,-0.111710,-0.113748,0.118644,1.831607,-0.396543,-0.122070,-0.250106,-0.450897,-0.332165,4.180299,...,0.127964,1.306674,-0.299451,0.140796,1.378727,-0.102556,-0.500327,0.202275,0.561808,0.526119
2,0.090462,-0.502083,-0.059683,-0.682626,-0.397647,-0.163870,-0.250740,-0.502466,-0.332611,-0.136593,...,-0.257329,-0.208183,-0.278110,-0.138206,-0.198377,-0.124777,-0.306719,0.043669,-0.174972,-0.154073
3,-0.143991,-0.349744,-0.189783,-0.485974,-0.391880,-0.164242,-0.250665,-0.300571,-0.328237,-0.139470,...,1.188030,1.033966,-0.058070,-0.055008,0.305962,-0.300284,0.023444,-0.226416,-0.159670,3.054452
4,-0.144492,-0.409669,-0.191357,0.003039,-0.373176,-0.164270,-0.250392,-0.486401,-0.285498,-0.156234,...,-0.310854,-0.153070,-0.169165,-0.305359,-0.268203,-0.142192,-0.343744,-0.228562,-0.197016,-0.247948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,-0.138860,-0.543929,-0.073434,-0.687548,-0.397524,-0.155174,-0.250739,-0.495441,-0.332608,-0.130375,...,3.978750,0.303906,0.092633,0.585478,0.257663,0.429276,0.494833,4.729899,4.630994,-0.338688
5019,4.783827,1.997961,0.333336,1.850811,6.342943,2.034023,-0.247243,5.612971,-0.324075,3.024356,...,-0.300507,4.943333,2.558837,2.642982,1.639358,1.499459,1.607354,2.046898,1.752810,3.238786
5020,-0.140555,-0.553483,-0.152443,0.561306,0.647902,0.057787,-0.250562,0.076153,-0.331908,-0.037337,...,-0.199389,0.039086,-0.373643,0.314501,-0.158225,0.533938,-0.514059,-0.218508,0.153822,0.420841
5021,-0.143869,0.203208,-0.176924,1.938559,0.238020,-0.164218,-0.167718,-0.086010,0.060280,-0.152157,...,-0.274437,-0.208020,-0.395044,-0.216634,-0.241317,-0.117355,-0.343776,-0.205415,0.074402,-0.248003


In [13]:
col_lab = list(range(0,1838))
X.set_axis(col_lab, axis="columns", inplace=True)
X

ValueError: Length mismatch: Expected axis has 1380 elements, new values have 1838 elements

In [8]:
import pandas as pd

snp_prvcs = pd.read_csv("/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/output/CAN_dataset_XY-named.csv", sep=',')
y = snp_prvcs['label']
y

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Encoded_sequences/CAN_dataset_XY-named.csv'

In [8]:
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [12]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score
model3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=7)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy_model3: %.4f" % (accuracy_score(y_test, pred3)))
print("F1_model3: %.4f" % (f1_score(y_test, pred3)))
print("MCC_model3: %.4f" % (matthews_corrcoef(y_test, pred3)))
print("ROC_AUC_model3: %.4f" % (roc_auc_score(y_test, pred3)))

Accuracy_model3: 0.7125
F1_model3: 0.6991
MCC_model3: 0.4268
ROC_AUC_model3: 0.7125


**PREDICTIONS**

In [13]:
abs_diff_plus_log_odds_scores_norm_pred = pd.read_csv("/content/drive/MyDrive/predictions/diff_plus_log_diff_norm_rslt01.csv", sep=',')
abs_diff_plus_log_odds_scores_norm_pred.head(), abs_diff_plus_log_odds_scores_norm_pred.shape

(          0         1         2             3         4         5         6  \
 0  0.000092  0.000036  0.000112  1.241336e-04  0.000317  0.000118  0.000061   
 1  0.000005  0.000017  0.000007  3.638554e-06  0.000059  0.000027  0.000007   
 2  0.000009  0.000012  0.000003  5.842000e-06  0.000006  0.000020  0.000016   
 3  0.000062  0.000005  0.000068  1.477122e-06  0.000219  0.000062  0.000034   
 4  0.000003  0.000366  0.000004  6.474913e-07  0.000087  0.000159  0.000059   
 
           7         8         9  ...      1828      1829      1830      1831  \
 0  0.000202  0.000199  0.000203  ...  0.035059  0.007059  0.005359  0.018831   
 1  0.000004  0.000003  0.000011  ...  0.028365  0.058806  0.004389  0.032639   
 2  0.000001  0.000002  0.000008  ...  0.004393  0.131629  0.029354  0.079405   
 3  0.000010  0.000013  0.000003  ...  0.027086  0.036587  0.005483  0.022586   
 4  0.000012  0.000013  0.000003  ...  0.021576  0.001458  0.001056  0.016763   
 
        1832      1833      18

In [14]:
X_pred=abs_diff_plus_log_odds_scores_norm_pred
X_pred

,0,1,2,3,4,5,6,7,8,9,...,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837
0,9.209999e-05,0.000036,0.000112,1.241336e-04,0.000317,0.000118,0.000061,2.017988e-04,1.992900e-04,2.030500e-04,...,0.035059,0.007059,0.005359,0.018831,0.020824,0.001042,0.008789,0.009422,0.014313,0.001904
1,4.894811e-06,0.000017,0.000007,3.638554e-06,0.000059,0.000027,0.000007,4.228680e-06,2.843773e-06,1.115881e-05,...,0.028365,0.058806,0.004389,0.032639,0.014551,0.007578,0.018783,0.034526,0.042478,0.070874
2,9.159562e-06,0.000012,0.000003,5.842000e-06,0.000006,0.000020,0.000016,1.078475e-06,1.977866e-06,7.560709e-06,...,0.004393,0.131629,0.029354,0.079405,0.023835,0.043884,0.030217,0.026512,0.027580,0.122850
3,6.240962e-05,0.000005,0.000068,1.477122e-06,0.000219,0.000062,0.000034,9.782356e-06,1.290666e-05,2.575602e-06,...,0.027086,0.036587,0.005483,0.022586,0.030726,0.000466,0.002374,0.018418,0.013104,0.056507
4,3.230773e-06,0.000366,0.000004,6.474913e-07,0.000087,0.000159,0.000059,1.249914e-05,1.267268e-05,2.728598e-06,...,0.021576,0.001458,0.001056,0.016763,0.006643,0.000758,0.002188,0.002559,0.001217,0.015825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9.456938e-07,0.000013,0.000002,2.140180e-06,0.000007,0.000003,0.000003,1.214433e-06,7.872253e-07,3.121889e-06,...,0.031456,0.075933,0.032650,0.021965,0.035114,0.022958,0.074309,0.097735,0.043841,0.008869
19996,5.900540e-06,0.000071,0.000004,5.396918e-07,0.000123,0.000072,0.000012,9.756119e-07,1.891346e-06,6.135175e-07,...,0.005715,0.077982,0.028584,0.006616,0.003699,0.023073,0.050146,0.055146,0.009801,0.008365
19997,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19998,4.977193e-04,0.000046,0.000091,3.348969e-05,0.000433,0.000235,0.000356,2.105481e-04,2.817596e-04,8.367252e-05,...,0.010997,0.013612,0.012717,0.002958,0.013707,0.002581,0.012380,0.016776,0.017415,0.003214


In [15]:
col_lab = list(range(0,1838))
X_pred.set_axis(col_lab, axis="columns", inplace=True)
X_pred

<ipython-input-15-cc47e8984377>:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X_pred.set_axis(col_lab, axis="columns", inplace=True)


,0,1,2,3,4,5,6,7,8,9,...,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837
0,9.209999e-05,0.000036,0.000112,1.241336e-04,0.000317,0.000118,0.000061,2.017988e-04,1.992900e-04,2.030500e-04,...,0.035059,0.007059,0.005359,0.018831,0.020824,0.001042,0.008789,0.009422,0.014313,0.001904
1,4.894811e-06,0.000017,0.000007,3.638554e-06,0.000059,0.000027,0.000007,4.228680e-06,2.843773e-06,1.115881e-05,...,0.028365,0.058806,0.004389,0.032639,0.014551,0.007578,0.018783,0.034526,0.042478,0.070874
2,9.159562e-06,0.000012,0.000003,5.842000e-06,0.000006,0.000020,0.000016,1.078475e-06,1.977866e-06,7.560709e-06,...,0.004393,0.131629,0.029354,0.079405,0.023835,0.043884,0.030217,0.026512,0.027580,0.122850
3,6.240962e-05,0.000005,0.000068,1.477122e-06,0.000219,0.000062,0.000034,9.782356e-06,1.290666e-05,2.575602e-06,...,0.027086,0.036587,0.005483,0.022586,0.030726,0.000466,0.002374,0.018418,0.013104,0.056507
4,3.230773e-06,0.000366,0.000004,6.474913e-07,0.000087,0.000159,0.000059,1.249914e-05,1.267268e-05,2.728598e-06,...,0.021576,0.001458,0.001056,0.016763,0.006643,0.000758,0.002188,0.002559,0.001217,0.015825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9.456938e-07,0.000013,0.000002,2.140180e-06,0.000007,0.000003,0.000003,1.214433e-06,7.872253e-07,3.121889e-06,...,0.031456,0.075933,0.032650,0.021965,0.035114,0.022958,0.074309,0.097735,0.043841,0.008869
19996,5.900540e-06,0.000071,0.000004,5.396918e-07,0.000123,0.000072,0.000012,9.756119e-07,1.891346e-06,6.135175e-07,...,0.005715,0.077982,0.028584,0.006616,0.003699,0.023073,0.050146,0.055146,0.009801,0.008365
19997,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19998,4.977193e-04,0.000046,0.000091,3.348969e-05,0.000433,0.000235,0.000356,2.105481e-04,2.817596e-04,8.367252e-05,...,0.010997,0.013612,0.012717,0.002958,0.013707,0.002581,0.012380,0.016776,0.017415,0.003214


In [16]:
predictions = model3.predict(xgboost.DMatrix(X_pred))

print(predictions)

/usr/local/lib/python3.10/dist-packages/xgboost/data.py:1037: UserWarning: Unknown data type: <class 'xgboost.core.DMatrix'>, trying to convert it to csr_matrix
  warnings.warn(


TypeError: Not supported type for data.<class 'xgboost.core.DMatrix'>